In [276]:
import pandas as pd

from scipy.stats import wilcoxon

In [277]:
dtc = pd.read_csv('effect_presence_dtc.csv')
dtr = pd.read_csv('effect_presence_dtr.csv')
rfc = pd.read_csv('effect_presence_rfc.csv')
rfr = pd.read_csv('effect_presence_rfr.csv')

In [278]:
def do_testing(pdf, metric):
    params = pdf['params'].drop_duplicates().values
    pdf0 = pdf[pdf['params'] == params[0]].sort_values('fold')
    pdf1 = pdf[pdf['params'] == params[1]].sort_values('fold')

    return pd.Series({
        'wilcoxon': wilcoxon(pdf0[metric], pdf1[metric], zero_method='zsplit').pvalue
    })

In [279]:
def summarize(data, metric):
    pd_wilcoxon = data.groupby('name')\
                    .apply(lambda x: do_testing(x, metric))\
                    .reset_index(drop=False)

    grouped = data.groupby(['name', 'params'])\
                    .agg({metric: 'mean'})\
                    .reset_index(drop=False)

    grouped['label'] = grouped.apply(lambda x: 'less' if eval(x['params'])['operator'] == '<' else 'leq', axis=1)

    pivoted = grouped.pivot(index='name', columns='label', values=metric)\
                .reset_index(drop=False)\
                .rename(columns={'leq': f'{metric}_leq', 'less': f'{metric}_l'})

    return pd.merge(pivoted, pd_wilcoxon, on=['name'])[['name', f'{metric}_leq', f'{metric}_l', 'wilcoxon']]

In [280]:
dtc = summarize(dtc, 'auc')
rfc = summarize(rfc, 'auc')
dtr = summarize(dtr, 'r2')
rfr = summarize(rfr, 'r2')

In [281]:
clas = pd.concat([dtc, rfc.drop(columns=['name'])], axis=1)
clas['name'] = clas['name'].apply(lambda x: x.replace('_', '-'))
clas.columns = pd.MultiIndex.from_tuples([
    ('Classification', '','name'),
    ('Classification', 'Decision Tree', 'AUC$_{leq}$'),
    ('Classification', 'Decision Tree', 'AUC$_{l}$'),
    ('Classification', 'Decision Tree', 'p'),
    ('Classification', 'Random Forest', 'AUC$_{leq}$'),
    ('Classification', 'Random Forest', 'AUC$_{l}$'),
    ('Classification', 'Random Forest', 'p'),
    ])
regr = pd.concat([dtr, rfr.drop(columns=['name'])], axis=1)
regr['name'] = regr['name'].apply(lambda x: x.replace('_', '-'))
regr.columns = pd.MultiIndex.from_tuples([
    ('Regression', '','name'),
    ('Regression', 'Decision Tree', 'r$^2_{leq}$'),
    ('Regression', 'Decision Tree', 'r$^2_{l}$'),
    ('Regression', 'Decision Tree', 'p'),
    ('Regression', 'Random Forest', 'r$^2_{leq}$'),
    ('Regression', 'Random Forest', 'r$^2_{l}$'),
    ('Regression', 'Random Forest', 'p'),
    ])

In [282]:
res = pd.concat([clas, regr], axis=1)

In [283]:
res

Classification                                                  \
                           Decision Tree                     Random Forest   
                      name   AUC$_{leq}$ AUC$_{l}$         p   AUC$_{leq}$   
0              abalone9-18      0.650018  0.650018  1.000000      0.827262   
1             appendicitis      0.712056  0.711997  0.928942      0.823626   
2                     bupa      0.626159  0.625188  0.517240      0.763548   
3         cleveland-0-vs-4      0.684516  0.688713  0.778081      0.972187   
4                   ecoli1      0.832220  0.832497  0.725537      0.954652   
5                   glass0      0.783446  0.783446  1.000000      0.931477   
6                 haberman      0.561040  0.561384  0.918423      0.668906   
7                hepatitis      0.650958  0.653467  0.024033      0.874199   
8             lymphography      0.899964  0.905038  0.596259      0.991580   
9             mammographic      0.786932  0.788242  0.021111      0.867026   
10                  monk-2      1.000000  1.000000  1.000000      1.000000   
11            new-thyroid1      0.934683  0.939710  0.002771      0.999437   
12    page-blocks-1-3-vs-4      0.957768  0.958268  0.998725      0.999592   
13                    pima      0.667564  0.667105  0.140981      0.822913   
14            poker-9-vs-7      0.596541  0.598713  0.000054      0.985730   
15                 saheart      0.585767  0.584992  0.043595      0.721527   
16        shuttle-6-vs-2-3      1.000000  1.000000  1.000000      1.000000   
17  winequality-red-3-vs-5      0.509359  0.509374  0.999432      0.741680   
18               wisconsin      0.936721  0.938521  0.001814      0.992831   
19    yeast-0-3-5-9-vs-7-8      0.641526  0.641538  0.544394      0.798224   

                                       Regression                          \
                                                  Decision Tree             
   AUC$_{l}$             p                   name   r$^2_{leq}$ r$^2_{l}$   
0   0.827222  2.159890e-02                airfoil      0.860597  0.860579   
1   0.823456  7.266978e-01               autoMPG6      0.763590  0.765539   
2   0.761703  7.827932e-07               baseball      0.405100  0.403743   
3   0.975642  5.983115e-14        cpu-performance      0.789304  0.785683   
4   0.954655  9.977234e-01           daily-demand      0.660710  0.661539   
5   0.931495  5.981432e-01               diabetes     -0.878408 -0.881574   
6   0.673240  1.030949e-09     excitation-current      0.999818  0.999818   
7   0.874157  2.757594e-01                  laser      0.922193  0.924267   
8   0.994313  4.561364e-03   maternal-health-risk      0.711601  0.710200   
9   0.867586  2.946361e-02           medical-cost      0.530427  0.530943   
10  0.999997  9.998578e-01                 o-ring      0.070045  0.324910   
11  0.999310  1.676531e-01                plastic      0.631103  0.629903   
12  0.999594  7.924234e-01  qsar-aquatic-toxicity      0.118513  0.120058   
13  0.823288  1.764932e-03  real-estate-valuation      0.458279  0.458620   
14  0.986722  2.930423e-02                  servo      0.465061  0.465097   
15  0.721258  8.950114e-02             slump-test      0.603482  0.600413   
16  1.000000  9.292232e-01        stock-portfolio      0.988020  0.988157   
17  0.741390  6.575842e-01                wankara      0.969651  0.969693   
18  0.993108  4.917077e-05                wsn-ale     -0.036866 -0.034827   
19  0.799399  5.183222e-12    yacht-hydrodynamics      0.992111  0.992111   

                                                        
                 Random Forest                          
               p   r$^2_{leq}$ r$^2_{l}$             p  
0   1.759729e-07      0.934265  0.934272  1.491365e-02  
1   1.265005e-09      0.872213  0.872158  5.051599e-01  
2   4.150450e-01      0.669715  0.668673  7.983798e-07  
3   7.118748e-03      0.860298  0.858250  1.706412e-06  
4   3.639243e-01      0.821434  0.822158  2.47

In [284]:
latex = res.to_latex(
    index=False,
    formatters={col: lambda x: f'\\bfseries {x:.3f}' if x < 0.05 else f'{x:.3f}'
                    for col in res.columns if col[-1] == 'p'} |
                {col: lambda x: f'{x:.3f}'
                    for col in res.columns if col[-1] not in ['p', 'name']},
    multicolumn_format='c'
)

In [285]:
tabular_string = latex[len('\\begin{tabular}{'): len('\\begin{tabular}{') + len(res.columns)]
tabular_string


'lrrrrrrlrrrrrr'

In [286]:
tab_clas = tabular_string[:7]
tab_regr = tabular_string[7:]

tab_clas_new = '@{\hspace{4pt}}'.join(tab_clas[:4]) + '@{\hspace{8pt}}' + '@{\hspace{4pt}}'.join(tab_clas[4:])
tab_regr_new = '@{\hspace{4pt}}'.join(tab_regr[:4]) + '@{\hspace{8pt}}' + '@{\hspace{4pt}}'.join(tab_regr[4:])

updated = tab_clas_new + '@{\hspace{4pt}}|@{\hspace{4pt}}' + tab_regr_new

In [287]:
latex = latex.replace(tabular_string, updated)

In [288]:
with open('tab_proof.tex', 'wt') as file:
    file.write(latex)

In [206]:
print(latex.replace('_', '-'))

\begin{tabular}{l@{\hspace{2pt}}r@{\hspace{2pt}}r@{\hspace{2pt}}r@{\hspace{2pt}}r@{\hspace{2pt}}r@{\hspace{2pt}}r@{\hspace{2pt}}l@{\hspace{2pt}}r@{\hspace{2pt}}r@{\hspace{2pt}}r@{\hspace{2pt}}r@{\hspace{2pt}}r@{\hspace{2pt}}r}
\toprule
\multicolumn{7}{c}{Classification} & \multicolumn{7}{c}{Regression} \\
 & \multicolumn{3}{c}{Decision Tree} & \multicolumn{3}{c}{Random Forest} &  & \multicolumn{3}{c}{Decision Tree} & \multicolumn{3}{c}{Random Forest} \\
name & AUC$-{leq}$ & AUC$-{l}$ & p & AUC$-{leq}$ & AUC$-{l}$ & p & name & r$^2-{leq}$ & r$^2-{l}$ & p & r$^2-{leq}$ & r$^2-{l}$ & p \\
\midrule
abalone9-18 & 0.650 & 0.650 & 1.000 & 0.827 & 0.827 & \bfseries 0.022 & airfoil & 0.861 & 0.861 & \bfseries 0.000 & 0.934 & 0.934 & \bfseries 0.015 \\
appendicitis & 0.712 & 0.712 & 0.929 & 0.824 & 0.823 & 0.727 & autoMPG6 & 0.764 & 0.766 & \bfseries 0.000 & 0.872 & 0.872 & 0.505 \\
bupa & 0.626 & 0.625 & 0.517 & 0.764 & 0.762 & \bfseries 0.000 & baseball & 0.405 & 0.404 & 0.415 & 0.670 & 0.669 

In [ ]:
dataset_map = {
    'lymphography-normal-fibrosis': 'lymphography',
    'stock-portfolio-performance': 'stock-portfolio'
}

In [165]:
print(res.style.apply(bold_p, axis=1).format({('Classification', 'Decision Tree', 'p'): lambda x: f"{x:,.3f}"}).to_latex(convert_css=True))

\begin{tabular}{llrrrrrrlrrrrrr}
 &  & \multicolumn{6}{r}{Classification} &  & \multicolumn{6}{r}{Regression} \\
 &  & \multicolumn{3}{r}{Decision Tree} & \multicolumn{3}{r}{Random Forest} &  & \multicolumn{3}{r}{Decision Tree} & \multicolumn{3}{r}{Random Forest} \\
 & name & AUC< & AUC<= & p & AUC< & AUC<= & p & namer & r2< & r2<= & p & r2< & r2<= & p \\
0 & abalone9_18 & 0.650018 & 0.650018 & 1.000 & 0.827262 & 0.827222 & 0.021599 & airfoil & 0.860597 & 0.860579 & 0.000000 & 0.934265 & 0.934272 & 0.014914 \\
1 & appendicitis & 0.712056 & 0.711997 & 0.929 & 0.823626 & 0.823456 & 0.726698 & autoMPG6 & 0.763590 & 0.765539 & 0.000000 & 0.872213 & 0.872158 & 0.505160 \\
2 & bupa & 0.626159 & 0.625188 & 0.517 & 0.763548 & 0.761703 & 0.000001 & baseball & 0.405100 & 0.403743 & 0.415045 & 0.669715 & 0.668673 & 0.000001 \\
3 & cleveland-0_vs_4 & 0.684516 & 0.688713 & 0.778 & 0.972187 & 0.975642 & 0.000000 & cpu_performance & 0.789304 & 0.785683 & 0.007119 & 0.860298 & 0.858250 & 0.000002 \\
4

In [71]:
pd.concat([clas, regr], axis=1).round(3)

NaN Classification                              \
                             NaN  Decision Tree               Random Forest   
                            name           AUC<  AUC<=      p          AUC<   
0                    abalone9_18          0.650  0.650  1.000         0.827   
1                   appendicitis          0.712  0.712  0.929         0.824   
2                           bupa          0.626  0.625  0.517         0.764   
3               cleveland-0_vs_4          0.685  0.689  0.778         0.972   
4                         ecoli1          0.832  0.832  0.726         0.955   
5                         glass0          0.783  0.783  1.000         0.931   
6                       haberman          0.561  0.561  0.918         0.669   
7                      hepatitis          0.651  0.653  0.024         0.874   
8   lymphography-normal-fibrosis          0.900  0.905  0.596         0.992   
9                   mammographic          0.787  0.788  0.021         0.867   
10                        monk-2          1.000  1.000  1.000         1.000   
11                  new_thyroid1          0.935  0.940  0.003         0.999   
12          page-blocks-1-3_vs_4          0.958  0.958  0.999         1.000   
13                          pima          0.668  0.667  0.141         0.823   
14                  poker-9_vs_7          0.597  0.599  0.000         0.986   
15                       saheart          0.586  0.585  0.044         0.722   
16              shuttle-6_vs_2-3          1.000  1.000  1.000         1.000   
17        winequality-red-3_vs_5          0.509  0.509  0.999         0.742   
18                     wisconsin          0.937  0.939  0.002         0.993   
19          yeast-0-3-5-9_vs_7-8          0.642  0.642  0.544         0.798   

                                          NaN    Regression                \
                                          NaN Decision Tree                 
    AUC<=      p                         name           r2<   r2<=      p   
0   0.827  0.022                      airfoil         0.861  0.861  0.000   
1   0.823  0.727                     autoMPG6         0.764  0.766  0.000   
2   0.762  0.000                     baseball         0.405  0.404  0.415   
3   0.976  0.000              cpu_performance         0.789  0.786  0.007   
4   0.955  0.998                 daily-demand         0.661  0.662  0.364   
5   0.931  0.598                     diabetes        -0.878 -0.882  0.791   
6   0.673  0.000           excitation_current         1.000  1.000  0.035   
7   0.874  0.276                        laser         0.922  0.924  0.000   
8   0.994  0.005         maternal_health_risk         0.712  0.710  0.001   
9   0.868  0.029                 medical_cost         0.530  0.531  0.000   
10  1.000  1.000                       o-ring         0.070  0.325  0.000   
11  0.999  0.168                      plastic         0.631  0.630  0.002   
12  1.000  0.792        qsar-aquatic-toxicity         0.119  0.120  0.533   
13  0.823  0.002        real_estate_valuation         0.458  0.459  0.862   
14  0.987  0.029                        servo         0.465  0.465  0.229   
15  0.721  0.090                   slump_test         0.603  0.600  0.747   
16  1.000  0.929  stock_portfolio_performance         0.988  0.988  0.261   
17  0.741  0.658                      wankara         0.970  0.970  0.020   
18  0.993  0.000                      wsn-ale        -0.037 -0.035  0.812   
19  0.799  0.000          yacht_hydrodynamics         0.992  0.992  1.000   

                                
   Random Forest                
             r2<   r2<=      p  
0          0.934  0.934  0.015  
1          0.872  0.872  0.505  
2          0.670  0.669  0.000  
3          0.860  0.858  0.000  
4          0.821  0.822  0.000  
5         -0.082 -0.083  0.108  
6          1.000  1.000  0.002  
7          0.963  0.963  0.142  
8          0.751  0.751  0.000  
9          0.694  0.694  0.494  
10        